In [2]:
import pandas as pd
# Import data
#Item_Fat_Content等を数字で置き換える。lightGBMはカテゴリ変数が扱えるので０始まりの数字に置き換えた。
df = pd.read_csv('train.csv', header=0).replace("low",0).replace("medium",1).replace("high",2).replace("RandD",0).replace("management",1).replace("hr",2).replace("IT",3).replace("technical",4).replace("product_mng",5).replace("accounting",6).replace("support",7).replace("sales",8).replace("marketing",9)
print(df.shape)
print(df.dtypes)
#欠損値を調べる
df.isnull().sum()

(10499, 11)
index                      int64
left                       int64
satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                      int64
salary                     int64
dtype: object


index                    0
left                     0
satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
promotion_last_5years    0
sales                    0
salary                   0
dtype: int64

In [3]:
df.head(3)

,index,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,10438,0,0.53,0.52,2,135,4,0,0,4,1
1,9236,0,0.77,0.53,5,256,3,0,0,6,1
2,818,1,0.89,0.79,3,149,2,0,0,7,1


In [4]:
df.sales.value_counts()

8    2865
4    1909
7    1550
3     870
5     628
9     601
0     558
6     536
2     534
1     448
Name: sales, dtype: int64

In [19]:
# Set dataframe
ID = df.loc[:, ['index']]
X  = df.drop(['index','left'], axis=1)
y  = df.loc[:, ['left']]

#標準化
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
sc = StandardScaler()
sc.fit(X)
X_std = sc.transform(X)
# n=6でK-Meansクラスタリング
km = KMeans(n_clusters=6,            
            init='random',           
            n_init=10,               
            max_iter=300,            
            tol=1e-04,               
            random_state=0)  
y_clus = km.fit_predict(X_std)
# cluster_idにクラスタリングの数字0-5を代入
X['cluster_id']=y_clus
X.head()
X.cluster_id.value_counts()
#X.groupby('cluster_id')[['satisfaction_level']].mean()
# クラスタリングのカテゴリに対しsatisfaction_levelで順位をつける。
y_value0 = np.where(y_clus == 0, 0.514440, y_clus)
y_value1 = np.where(y_value0 == 1, 0.647480, y_value0)
y_value2 = np.where(y_value1 == 2, 0.171819, y_value1)
y_value3 = np.where(y_value2 == 3, 0.658440, y_value2)
y_value4 = np.where(y_value3 == 4, 0.762524, y_value3)
y_value5 = np.where(y_value4 == 5, 0.660154, y_value4)

#cluster_id_valueにsatisfaction_levelの順位を代入
X['cluster_id_value']=y_value5
X.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,cluster_id,cluster_id_value
0,0.53,0.52,2,135,4,0,0,4,1,2,0.171819
1,0.77,0.53,5,256,3,0,0,6,1,5,0.660154
2,0.89,0.79,3,149,2,0,0,7,1,5,0.660154
3,0.64,0.63,3,156,6,1,0,7,0,3,0.658440
4,0.98,0.74,4,151,3,0,0,8,1,5,0.660154


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# split data for crossvalidation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#ここからlightbgm
import lightgbm as lgb
import numpy as np

lgb_train = lgb.Dataset(X_train, y_train.as_matrix().ravel())
lgb_eval = lgb.Dataset(X_test, y_test.as_matrix().ravel(), reference=lgb_train)
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'metric': {'multi_logloss'},
        'num_class': 3,
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 100,
        'verbose': 0
}
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred = np.argmax(y_pred, axis=1)

/Users/ryoshi/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 0.594698
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's multi_logloss: 0.524339
[3]	valid_0's multi_logloss: 0.465676
[4]	valid_0's multi_logloss: 0.416122
[5]	valid_0's multi_logloss: 0.373897
[6]	valid_0's multi_logloss: 0.337416
[7]	valid_0's multi_logloss: 0.305798
[8]	valid_0's multi_logloss: 0.278462
[9]	valid_0's multi_logloss: 0.254648
[10]	valid_0's multi_logloss: 0.233591
[11]	valid_0's multi_logloss: 0.215251
[12]	valid_0's multi_logloss: 0.198724
[13]	valid_0's multi_logloss: 0.184268
[14]	valid_0's multi_logloss: 0.171508
[15]	valid_0's multi_logloss: 0.159555
[16]	valid_0's multi_logloss: 0.149492
[17]	valid_0's multi_logloss: 0.140461
[18]	valid_0's multi_logloss: 0.132343
[19]	valid_0's multi_logloss: 0.125289
[20]	valid_0's multi_logloss: 0.118568
[21]	valid_0's multi_logloss: 0.112156
[22]	valid_0's multi_logloss: 0.10649
[23]	valid_0's multi_logloss: 0.102189
[24]	valid_0's multi_logloss: 0.0981195
[25]	valid_

In [8]:
#accuracy scoreの算出
from sklearn.metrics import accuracy_score
accu_test = accuracy_score(y_test, y_pred)
print(accu_test)
#以前は0.98380952381。これでいくと0.987142857143


0.987142857143


テストデータの読み込み

In [9]:
# read final_hr_analysis_test.csv
df_test = pd.read_csv('test.csv', header=0).replace("low",0).replace("medium",1).replace("high",2).replace("RandD",0).replace("management",1).replace("hr",2).replace("IT",3).replace("technical",4).replace("product_mng",5).replace("accounting",6).replace("support",7).replace("sales",8).replace("marketing",9)

In [10]:
df_test.head(3)

,index,left,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,1670,NaN,0.44,0.57,2,141,3,0,0,5,1
1,13378,NaN,0.55,0.96,3,194,3,0,0,5,1
2,10233,NaN,0.72,0.67,5,210,2,0,0,1,1


In [11]:
# delete index and left columns
X_testfile  = df_test.drop(['index','left'], axis=1)
ID_testfile = df_test.loc[:, ['index']]

In [12]:
X_testfile.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary
0,0.44,0.57,2,141,3,0,0,5,1
1,0.55,0.96,3,194,3,0,0,5,1
2,0.72,0.67,5,210,2,0,0,1,1
3,0.96,0.75,4,177,2,0,0,3,0
4,0.96,0.54,3,198,3,0,0,7,0


In [13]:
y_clus_test = km.fit_predict(X_testfile)
X_testfile['cluster_id']=y_clus_test

y_clus_test0 = np.where(y_clus_test == 0, 0.514440, y_clus_test)
y_clus_test1 = np.where(y_clus_test0 == 1, 0.647480, y_clus_test0)
y_clus_test2 = np.where(y_clus_test1 == 2, 0.171819, y_clus_test1)
y_clus_test3 = np.where(y_clus_test2 == 3, 0.658440, y_clus_test2)
y_clus_test4 = np.where(y_clus_test3 == 4, 0.762524, y_clus_test3)
y_clus_test5 = np.where(y_clus_test4 == 5, 0.660154, y_clus_test4)
X_testfile['cluster_id_value']=y_clus_test5

In [14]:
X_testfile.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,cluster_id,cluster_id_value
0,0.44,0.57,2,141,3,0,0,5,1,3,0.658440
1,0.55,0.96,3,194,3,0,0,5,1,4,0.762524
2,0.72,0.67,5,210,2,0,0,1,1,0,0.514440


In [15]:
X_train.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,cluster_id,cluster_id_value
2847,0.90,0.98,4,264,6,0,0,5,1,5,0.660154
5790,0.38,0.50,2,139,3,0,0,0,0,2,0.171819
10134,0.87,0.84,4,264,6,0,0,7,0,0,0.514440


In [16]:
y_pred_test = gbm.predict(X_testfile, num_iteration=gbm.best_iteration)
#y_pred_test = np.argmax(y_pred_test, axis=1)
y_pred_test[:3]


array([[  1.82749678e-02,   9.81725032e-01,   1.29014106e-16],
       [  9.98652133e-01,   1.34786740e-03,   3.53382190e-17],
       [  9.98105805e-01,   1.89419520e-03,   4.18807656e-17]])

In [17]:
#保存
import numpy
numpy.savetxt("result_lightgbm_clust.csv", y_pred_test, delimiter=",",fmt='%.6f')
#numpy.savetxt("result_proba.csv", Y_pred_proba, delimiter=",",fmt='%.f')

In [18]:
#おまじない
testfile_data = df_test.values
#CSVに保存
import csv
with open("032518_predict_result_lightgbm_clustdata.csv", "w") as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(["Id", "left"])
    for pid, survived in zip(testfile_data[:,0].astype(int), y_pred_test[:,1].astype(numpy.float64)):
        writer.writerow([pid, survived])